In [ ]:
import cv2
import numpy as np
import random

In [2]:
src_img = cv2.imread('Carnival.png')
hidden_img = cv2.imread('hidden.jpg')

In [3]:
height_src, width_src, channels_src = src_img.shape
print(height_src, width_src, channels_src)

408 620 3


In [4]:
height_hidden, width_hidden, channels_hidden = hidden_img.shape
print(height_hidden, width_hidden, channels_hidden)

200 283 3


In [5]:
# Encryption function
def encrypt(source,hidden):
    if source.shape[0] < hidden.shape[0]:
        height = source.shape[0]
        width = source.shape[1]
    else:
        height = hidden.shape[0]
        width = hidden.shape[1]
      
    for i in range(height):
        for j in range(width):
            for l in range(3):
                  
                # v1 and v2 are 8-bit pixel values
                # of source and hidden images respectively
                v1 = format(source[i][j][l], '08b')
                v2 = format(hidden[i][j][l], '08b')
                  
                # Taking 4 MSBs of each image
                v3 = v1[:4] + v2[:4] 
                  
                source[i][j][l]= int(v3, 2)
      
    return source

In [6]:
# Decryption function
def decrypt(encrypted):
    height = encrypted.shape[0]
    width = encrypted.shape[1]
      
    hidden = np.zeros((height, width, 3), np.uint8)
      
    for i in range(height):
        for j in range(width):
            for l in range(3):
                v1 = format(encrypted[i][j][l], '08b')
                v2 = v1[:4] + chr(random.randint(0, 1)+48) * 4
                v3 = v1[4:] + chr(random.randint(0, 1)+48) * 4
                  
                # Appending data to img1 and img2
                hidden[i][j][l]= int(v3, 2)
      
    return hidden  

# smaller size

In [7]:
hidden_img_small = cv2.resize(hidden_img, (width_src - 200, height_src - 200))

In [8]:
height_hidden_small, width_hidden_small, channels_hidden_small = hidden_img_small.shape
print(height_hidden_small, width_hidden_small, channels_hidden_small)

208 420 3


# encrypt

In [9]:
encrypted_small = encrypt(src_img, hidden_img_small)
cv2.imwrite('encrypted_small.png', encrypted_small)

True

# decrypt

In [10]:
hidden_small = decrypt(encrypted_small)
cv2.imwrite('decrypted_small.png', hidden_small)

True

# same size

In [11]:
hidden_img_same = cv2.resize(hidden_img, (width_src, height_src))

In [12]:
height_hidden_same, width_hidden_same, channels_hidden_same = hidden_img_same.shape
print(height_hidden_same, width_hidden_same, channels_hidden_same)

408 620 3


# encrypt

In [13]:
encrypted_same = encrypt(src_img, hidden_img_same)
cv2.imwrite('encrypted_same.png', encrypted_same)

True

# decrypt

In [14]:
hidden_same = decrypt(encrypted_same)
cv2.imwrite('decrypted_same.png', hidden_same)

True

# larger size

In [15]:
hidden_img_large = cv2.resize(hidden_img, (width_src + 200, height_src + 200))

In [16]:
height_hidden_large, width_hidden_large, channels_hidden_large = hidden_img_large.shape
print(height_hidden_large, width_hidden_large, channels_hidden_large)

608 820 3


# encrypt 

In [17]:
encrypted_large = encrypt(src_img, hidden_img_large)
cv2.imwrite('encrypted_large.png', encrypted_large)

True

# decrypt

In [18]:
hidden_large = decrypt(encrypted_large)
cv2.imwrite('decrypted_large.png', hidden_large)

True

# Task 2b

In [26]:
from PIL import Image
import numpy as np


# Encryption Functions

def encode_str(str_input):
    output = []
    length=len(str_input)
    for x in range(length):
        resLet = ord(str_input[x])
        output.append(resLet)
    return output


def hash_enc_str(enc_str):
    
    for x in range(len(enc_str)):
        if enc_str[x] == 32:
            enc_str[x] = 31
        elif enc_str[x] == 46:
            enc_str[x] = 30
        elif enc_str[x] == 44:
            enc_str[x] = 29
        else:
            enc_str[x] = enc_str[x] % 96
    return enc_str

def encrypt_msg(input):
    small_ltr = input.lower()
    input_arr = list(small_ltr)
    print({'Small Letter Array': input_arr})
    enc_str = encode_str(input_arr)
    print({'Encoded String': enc_str})
    final_out = hash_enc_str(enc_str)
    print({'Hashed String': final_out})
    return final_out


def decimalToBinary(n):
    return bin(n).replace("0b", "")

def modifyBit( n,  p,  b):
    mask = 1 << p
    return (n & ~mask) | ((b << p) & mask)

def imageManipulation(img, input):

    # img = cv2.imread("lena.png")
    # blue, green, red = cv2.split(img)
    photo= Image.open(img).convert('RGB')
    newImgArr = np.asarray(photo).copy()
    img_shape = newImgArr.shape
    print({'Shape': img_shape})

    flat_img = newImgArr.reshape(newImgArr.shape[0]*newImgArr.shape[1], newImgArr.shape[2] )

    # print(newImgArr)
    count = 0
    for i in range(0, len(flat_img)-1, 2):
        letter_binary = decimalToBinary(input[count])
        if len(letter_binary) == 1:
            temp = '0000'
            letter_binary = temp + letter_binary
        elif len(letter_binary) == 2:
            temp = '000'
            letter_binary = temp + letter_binary
        elif len(letter_binary) == 3:
            temp = '00'
            letter_binary = temp + letter_binary
        elif len(letter_binary) == 4:
            temp = '0'
            letter_binary = temp + letter_binary

        # print(letter_binary)
        # print("\n")

        flat_img[i][0] = modifyBit(flat_img[i][0], 0, int(letter_binary[0],10))
        flat_img[i][1] = modifyBit(flat_img[i][1], 0, int(letter_binary[1],10))
        flat_img[i+1][0] = modifyBit(flat_img[i+1][0], 0, int(letter_binary[2],10))
        flat_img[i+1][1] = modifyBit(flat_img[i+1][1], 0, int(letter_binary[3],10))
        flat_img[i+1][2] = modifyBit(flat_img[i+1][2], 0, int(letter_binary[4],10))
        count = count + 1


        if len(input) == count:
            flat_img[i+2][0] = modifyBit(flat_img[i+1][0], 0, 0)
            flat_img[i+2][1] = modifyBit(flat_img[i+1][1], 0, 0)
            flat_img[i+3][0] = modifyBit(flat_img[i+2][0], 0, 0)
            flat_img[i+3][1] = modifyBit(flat_img[i+2][1], 0, 0)
            flat_img[i+3][2] = modifyBit(flat_img[i+2][2], 0, 0)
            break;
    newImgArr = flat_img.reshape(img_shape)
    return Image.fromarray(newImgArr)


def encodeMsgintoImg(img, msg):
    print({'Size of Message': len(msg)})
    encMsg = encrypt_msg(msg)
    encImg = imageManipulation(img, encMsg)
    encImg.save('enc_img.png')
    #return encImg


def decimalToBinary(n):
    return bin(n).replace("0b", "")


#decryption Functions

def hash_dec(input):
    for x in range(len(input)):
        if input[x] == 31:
            input[x] = 32
        elif input[x] == 30:
            input[x] = 46
        elif input[x] == 29:
            input[x] = 44
        else:
            input[x] = input[x] + 96
    
    print({'De-Hasheed': input})
    return input

def decodeString(string_input):
    decode=[]
    outLength=len(string_input)
    for i in range(outLength):
        resLet = string_input[i]
        decode.append(chr(resLet))
        
    print({'Decoded': decode})
    return decode

def decrypt_msg(encMsg):
    dehashed_msg = hash_dec(encMsg)
    decode_msg = decodeString(dehashed_msg)
    return decode_msg


def extractingMessage(img):

    photo= Image.open(img)
    newImgArr = np.asarray(photo).copy()
    img_shape = newImgArr.shape
    print({'Shape': img_shape})

    flat_img = newImgArr.reshape(newImgArr.shape[0]*newImgArr.shape[1], newImgArr.shape[2])

    output = []

    for i in range(0, len(flat_img)-1, 2):
        temp = decimalToBinary(flat_img[i][0])
        number = temp[len(temp)-1]
        temp = decimalToBinary(flat_img[i][1])
        number = number + temp[len(temp)-1]
        temp = decimalToBinary(flat_img[i+1][0])
        number = number + temp[len(temp)-1]
        temp = decimalToBinary(flat_img[i+1][1])
        number = number + temp[len(temp)-1]
        temp = decimalToBinary(flat_img[i+1][2])
        number = number + temp[len(temp)-1]

        value = int(number,2)
        #print(value)

        if value == 0:
            break;
        output.append(value)

    print({'Image Data': output})
    return output

def decodeMsgFromImg(img):
    
    encMsg = extractingMessage(img)
    decMsg = decrypt_msg(encMsg)
    output = ""
    #return output.join(decMsg)
    print({'Output': output.join(decMsg)})


# Encrypting One Word

In [27]:
img = "Carnival.png"
msg = "ephemeral"
encodeMsgintoImg(img,msg)

{'Size of Message': 9}
{'Small Letter Array': ['e', 'p', 'h', 'e', 'm', 'e', 'r', 'a', 'l']}
{'Encoded String': [101, 112, 104, 101, 109, 101, 114, 97, 108]}
{'Hashed String': [5, 16, 8, 5, 13, 5, 18, 1, 12]}
{'Shape': (408, 620, 3)}


# Decrypting That One Word

In [28]:
img = "enc_img.png"
decodeMsgFromImg(img)

{'Shape': (408, 620, 3)}
{'Image Data': [5, 16, 8, 5, 13, 5, 18, 1, 12]}
{'De-Hasheed': [101, 112, 104, 101, 109, 101, 114, 97, 108]}
{'Decoded': ['e', 'p', 'h', 'e', 'm', 'e', 'r', 'a', 'l']}
{'Output': 'ephemeral'}


# Encrypting Phrase

In [30]:
img = "Carnival.png"
msg = "We are what we repeatedly do"
encodeMsgintoImg(img,msg)

{'Size of Message': 28}
{'Small Letter Array': ['w', 'e', ' ', 'a', 'r', 'e', ' ', 'w', 'h', 'a', 't', ' ', 'w', 'e', ' ', 'r', 'e', 'p', 'e', 'a', 't', 'e', 'd', 'l', 'y', ' ', 'd', 'o']}
{'Encoded String': [119, 101, 32, 97, 114, 101, 32, 119, 104, 97, 116, 32, 119, 101, 32, 114, 101, 112, 101, 97, 116, 101, 100, 108, 121, 32, 100, 111]}
{'Hashed String': [23, 5, 31, 1, 18, 5, 31, 23, 8, 1, 20, 31, 23, 5, 31, 18, 5, 16, 5, 1, 20, 5, 4, 12, 25, 31, 4, 15]}
{'Shape': (408, 620, 3)}


# Decrypting Phrase

In [31]:
img = "enc_img.png"
decodeMsgFromImg(img)

{'Shape': (408, 620, 3)}
{'Image Data': [23, 5, 31, 1, 18, 5, 31, 23, 8, 1, 20, 31, 23, 5, 31, 18, 5, 16, 5, 1, 20, 5, 4, 12, 25, 31, 4, 15]}
{'De-Hasheed': [119, 101, 32, 97, 114, 101, 32, 119, 104, 97, 116, 32, 119, 101, 32, 114, 101, 112, 101, 97, 116, 101, 100, 108, 121, 32, 100, 111]}
{'Decoded': ['w', 'e', ' ', 'a', 'r', 'e', ' ', 'w', 'h', 'a', 't', ' ', 'w', 'e', ' ', 'r', 'e', 'p', 'e', 'a', 't', 'e', 'd', 'l', 'y', ' ', 'd', 'o']}
{'Output': 'we are what we repeatedly do'}


# Encrypting Sentence

In [32]:
img = "Carnival.png"
msg = "the test did not produce any meaningful results"
encodeMsgintoImg(img,msg)

{'Size of Message': 47}
{'Small Letter Array': ['t', 'h', 'e', ' ', 't', 'e', 's', 't', ' ', 'd', 'i', 'd', ' ', 'n', 'o', 't', ' ', 'p', 'r', 'o', 'd', 'u', 'c', 'e', ' ', 'a', 'n', 'y', ' ', 'm', 'e', 'a', 'n', 'i', 'n', 'g', 'f', 'u', 'l', ' ', 'r', 'e', 's', 'u', 'l', 't', 's']}
{'Encoded String': [116, 104, 101, 32, 116, 101, 115, 116, 32, 100, 105, 100, 32, 110, 111, 116, 32, 112, 114, 111, 100, 117, 99, 101, 32, 97, 110, 121, 32, 109, 101, 97, 110, 105, 110, 103, 102, 117, 108, 32, 114, 101, 115, 117, 108, 116, 115]}
{'Hashed String': [20, 8, 5, 31, 20, 5, 19, 20, 31, 4, 9, 4, 31, 14, 15, 20, 31, 16, 18, 15, 4, 21, 3, 5, 31, 1, 14, 25, 31, 13, 5, 1, 14, 9, 14, 7, 6, 21, 12, 31, 18, 5, 19, 21, 12, 20, 19]}
{'Shape': (408, 620, 3)}


# Decrypting Sentence

In [33]:
img = "enc_img.png"
decodeMsgFromImg(img)

{'Shape': (408, 620, 3)}
{'Image Data': [20, 8, 5, 31, 20, 5, 19, 20, 31, 4, 9, 4, 31, 14, 15, 20, 31, 16, 18, 15, 4, 21, 3, 5, 31, 1, 14, 25, 31, 13, 5, 1, 14, 9, 14, 7, 6, 21, 12, 31, 18, 5, 19, 21, 12, 20, 19]}
{'De-Hasheed': [116, 104, 101, 32, 116, 101, 115, 116, 32, 100, 105, 100, 32, 110, 111, 116, 32, 112, 114, 111, 100, 117, 99, 101, 32, 97, 110, 121, 32, 109, 101, 97, 110, 105, 110, 103, 102, 117, 108, 32, 114, 101, 115, 117, 108, 116, 115]}
{'Decoded': ['t', 'h', 'e', ' ', 't', 'e', 's', 't', ' ', 'd', 'i', 'd', ' ', 'n', 'o', 't', ' ', 'p', 'r', 'o', 'd', 'u', 'c', 'e', ' ', 'a', 'n', 'y', ' ', 'm', 'e', 'a', 'n', 'i', 'n', 'g', 'f', 'u', 'l', ' ', 'r', 'e', 's', 'u', 'l', 't', 's']}
{'Output': 'the test did not produce any meaningful results'}
